In [ ]:
#r "nuget: FSharp.Data"
#r "nuget: FSharp.Stats"
#r "nuget: Newtonsoft.JSON"
#r "nuget: Plotly.NET, 2.0.0-preview.6"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.6"


[![Binder](https://fslab.org/images/badge-binder.svg)](https://mybinder.org/v2/gh/fslaborg/fslaborg.github.io/gh-pages?filepath=content/tutorials/006_savitzky_golay_temperature.ipynb)&emsp;
[![Script](https://fslab.org/images/badge-script.svg)](https://fslab.org/content/tutorials/006_savitzky_golay_temperature.fsx)&emsp;
[![Notebook](https://fslab.org/images/badge-notebook.svg)](https://fslab.org/content/tutorials/006_savitzky_golay_temperature.ipynb)


# Smoothing data with the Savitzky-Golay filter

_Summary:_ This tutorial demonstrates how to access a public dataset for temperature data with [FSharp.Data](https://fsprojects.github.io/FSharp.Data/), how to smooth the data points with 
the Savitzky-Golay filter from [FSharp.Stats](https://fslab.org/FSharp.Stats/) and finally how to visualize the results with [Plotly.NET](https://plotly.net).

## Introduction: 

The Savitzky-Golay is a type of low-pass filter, particularly suited for smoothing noisy data. The main idea behind this approach is to make for each point a 
least-square fit with a polynomial of high order over a odd-sized window centered at the point. One advantage of the Savitzky-Golay filter is that portions 
of high frequencies are not simply cut off, but are preserved due to the polynomial regression. This allows the filter to preserve properties of the distribution 
such as relative maxima, minima, and dispersion, which are usually distorted by flattening or shifting by conventional methods such as moving average.

This is useful when trying to identify general trends in highly fluctuating data sets, or to smooth out noise to improve the ability to find minima and maxima of the data trend.
To showcase this we will plot a temperature dataset from the "[Deutscher Wetterdienst](https://www.dwd.de/DE/leistungen/klimadatendeutschland/klimadatendeutschland.html)", 
a german organization for climate data. We will do this for both the original data points and a smoothed version.

![windowed polynomial regression](https://upload.wikimedia.org/wikipedia/commons/8/89/Lissage_sg3_anim.gif)

The image shows the moving window for polynomial regression used in the Savitzky-Golay filter [@wikipedia](https://upload.wikimedia.org/wikipedia/commons/8/89/Lissage_sg3_anim.gif)


## Referencing packages

```fsharp
// Packages hosted by the Fslab community
#r "nuget: FSharp.Stats"
// third party .net packages 
#r "nuget: FSharp.Data"
#r "nuget: Plotly.NET, 2.0.0-preview.6"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.6"
```


## Loading data

We will start by retrieving the data. This is done with the [FSharp.Data](https://fsprojects.github.io/FSharp.Data/) package 
and will return a single string in the original format.



In [3]:
// Get data from Deutscher Wetterdienst
// Explanation for Abbreviations: https://www.dwd.de/DE/leistungen/klimadatendeutschland/beschreibung_tagesmonatswerte.html
let rawData = FSharp.Data.Http.RequestString @"https://raw.githubusercontent.com/fslaborg/datasets/main/data/WeatherDataAachen-Orsbach_daily_1year.txt"

// print first 1000 characters to console.
rawData.[..1000] |> printfn "%s"


<pre>                 Tageswerte der Station 10505 Aachen-Orsbach                                                STAT JJJJMMDD QN     TG     TN     TM     TX    RFM     FM     FX     SO     NM     RR     PM----- -------- -- ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------10505 20210728  1   13.5   14.4   17.7   21.5   73.3    3.0   13.9    4.9    6.4    5.5  983.810505 20210727  1   14.3   15.2   17.9   23.4   81.6    3.0   10.1    3.8    6.1    1.2  984.810505 20210726  1   14.8   16.0   18.5   22.6   79.3    3.0    9.8    5.2    6.7    1.4  983.710505 20210725  1   11.5   14.2   18.7   25.2   79.4    2.0   11.5    7.9           6.1  981.710505 20210724  1   12.4   13.8   17.9   21.8   83.8    2.0   10.2    0.3    7.5    2.8  982.410505 20210723  1    9.1   11.8   18.5   24.8   68.7    2.0    7.4   13.8    1.3    0.0  989.310505 20210722  1   10.1   12.9   19.1   24.1   67.9    2.0    5.8   11.0    1.1    0.0  994.210505 20210721  1    8.4   11.1

Currently the data set is not in a format, that is easily parsable. Normally you would try to use 
the Deedle package to read in the data into a [Deedle](https://fslab.org/Deedle/) data frame. As this is not possible here, we will do some ugly formatting.

## Data Formatting/Parsing



In [4]:
open System
open System.Text.RegularExpressions

/// Tuple of 4 data arrays representing the measured temperature for over a year.
let processedData = 
    // First separate the huge string in lines
    rawData.Split([|'\n'|], StringSplitOptions.RemoveEmptyEntries)
    // Skip the first 5 rows until the real data starts, also skip the last row (length-2) to remove a "</pre>" at the end
    |> fun arr -> arr.[5..arr.Length-2]
    |> Array.map (fun data -> 
        // Regex pattern that will match groups of whitespace
        let whitespacePattern = @"\s+"
        // This is needed to tell regex to replace hits with a tabulator
        let matchEval = MatchEvaluator(fun _ -> @"\t" )
        // The original data columns are separated by different amounts of whitespace.
        // Therefore, we need a flexible string parsing option to replace any amount of whitespace with a single tabulator.
        // This is done with the regex pattern above and the fsharp core library "System.Text.RegularExpressions" 
        let tabSeparated = Regex.Replace(data, whitespacePattern, matchEval)
        tabSeparated
        // Split each row by tabulator will return rows with an equal amount of values, which we can access.
        |> fun dataStr -> dataStr.Split([|@"\t"|], StringSplitOptions.RemoveEmptyEntries)
        |> fun dataArr -> 
            // Second value is the date of measurement, which we will parse to the DateTime type
            DateTime.ParseExact(dataArr.[1], "yyyyMMdd", Globalization.CultureInfo.InvariantCulture),
            // 5th value is minimal temperature at that date.
            float dataArr.[4],
            // 6th value is average temperature over 24 timepoints at that date.
            float dataArr.[5],
            // 7th value is maximal temperature at that date.
            float dataArr.[6]
    )
    // Sort by date
    |> Array.sortBy (fun (day,tn,tm,tx) -> day)
    // Unzip the array of value tuples, to make the different values easier accessible
    |> fun arr -> 
        arr |> Array.map (fun (day,tn,tm,tx) -> day.ToShortDateString()),
        arr |> Array.map (fun (day,tn,tm,tx) -> tm),
        arr |> Array.map (fun (day,tn,tm,tx) -> tx),
        arr |> Array.map (fun (day,tn,tm,tx) -> tn)


([|"03/16/2020"; "03/17/2020"; "03/18/2020"; "03/19/2020"; "03/20/2020";   "03/21/2020"; "03/22/2020"; "03/23/2020"; "03/24/2020"; "03/25/2020";   "03/26/2020"; "03/27/2020"; "03/28/2020"; "03/29/2020"; "03/30/2020";   "03/31/2020"; "04/01/2020"; "04/02/2020"; "04/03/2020"; "04/04/2020";   "04/05/2020"; "04/06/2020"; "04/07/2020"; "04/08/2020"; "04/09/2020";   "04/10/2020"; "04/11/2020"; "04/12/2020"; "04/13/2020"; "04/14/2020";   "04/15/2020"; "04/16/2020"; "04/17/2020"; "04/18/2020"; "04/19/2020";   "04/20/2020"; "04/21/2020"; "04/22/2020"; "04/23/2020"; "04/24/2020";   "04/25/2020"; "04/26/2020"; "04/27/2020"; "04/28/2020"; "04/29/2020";   "04/30/2020"; "05/01/2020"; "05/02/2020"; "05/03/2020"; "05/04/2020";   "05/05/2020"; "05/06/2020"; "05/07/2020"; "05/08/2020"; "05/09/2020";   "05/10/2020"; "05/11/2020"; "05/12/2020"; "05/13/2020"; "05/14/2020";   "05/15/2020"; "05/16/2020"; "05/17/2020"; "05/18/2020"; "05/19/2020";   "05/20/2020"; "05/21/2020"; "05/22/2020"; "05/23/2020"; "05/2

## Exploring the data set with Plotly.NET

Next we create a create chart function with [Plotly.NET](https://plotly.net) to produce a visual representation of our data set.



In [6]:
open Plotly.NET

// Because our data set is already rather wide we want to move the legend from the right side of the plot
// to the right center. As this function is not defined for fsharp we will use the underlying js bindings (https://plotly.com/javascript/legend/#positioning-the-legend-inside-the-plot).
// Declarative style in F# using underlying DynamicObj
// https://plotly.net/#Declarative-style-in-F-using-the-underlying
let legend = 
    let tmp = Legend()
    tmp?yanchor <- "top"
    tmp?y <- 0.99
    tmp?xanchor <- "left"
    tmp?x <- 0.5
    tmp

/// This function will take 'processedData' as input and return a range chart with a line for the average temperature
/// and a different colored area for the range between minimal and maximal temperature at that date.
let createTempChart (days,tm,tmUpper,tmLower) =
    Chart.Range(
        // data arrays
        days, tm, tmUpper, tmLower,
        StyleParam.Mode.Lines_Markers,
        Color="#3D1244",
        RangeColor="#F99BDE",
        // Name for line in legend
        Name="Average temperature over 24 timepoints each day",
        // Name for lower point when hovering over chart
        LowerName="Min temp",
        // Name for upper point when hovering over chart
        UpperName="Max temp"
    )
    // Configure the chart with the legend from above
    |> Chart.withLegend legend
    // Add name to y axis
    |> Chart.withY_AxisStyle("daily temperature [°C]")
    |> Chart.withSize (1000.,600.)

/// Chart for original data set 
let rawChart =
    processedData 
    |> createTempChart


<!-- Plotly chart will be drawn inside this DIV -->

As you can see the data looks chaotic and is difficult to analyze. Trends are hidden in daily 
temperature fluctuations and correlating events with temperature can get difficult. So next we want to
smooth the data to clearly see temperature trends.

## Savitzky-Golay filter

We will use the `Signal.Filtering.savitzkyGolay` function from [FSharp.Stats](https://fslab.org/FSharp.Stats/).

Parameters:

- windowSize (`int`) the length of the window. Must be an odd integer number.
- order (`int`) the order of the polynomial used in the filtering. Must be less then `windowSize` - 1.
- deriv (`int`) the order of the derivative to compute (default = 0 means only smoothing)
- rate (`int`) this factor will influence amplitude when using Savitzky-Golay for derivation
- data (`float array`) the values of the time history of the signal.




In [8]:
open FSharp.Stats

let smootheTemp ws order (days,tm,tmUpper,tmLower) =
    let tm' = Signal.Filtering.savitzkyGolay ws order 0 1 tm
    let tmUpper' = Signal.Filtering.savitzkyGolay ws order 0 1 tmUpper
    let tmLower' = Signal.Filtering.savitzkyGolay ws order 0 1 tmLower
    days,tm',tmUpper',tmLower'

let smoothedChart =
    processedData
    |> smootheTemp 31 4
    |> createTempChart 


<!-- Plotly chart will be drawn inside this DIV -->